Loading the data

id: household

Choices:  
- heinz41  
- heinz32  
- heinz28  
- hunts32  

variables  
- disp (1/0)  
- feat (1/0)  
- price (Log?)  

In [1]:
import numpy as np
import pandas as pd
df = pd.read_pickle("data.pkl") 
df.head()

,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz28,2
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz28,2
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28,2


Defining the likelihood function using numba and numpy arrays and multiple draws

In [2]:
import numpy as np
import pandas as pd
import datetime
import time
from scipy.optimize import minimize
from numba import jit, prange

@jit(nopython=True, parallel=True)
def likelihood(c, data, draws, verbose=False):
    #print("evaluation likelihood")
    n_r = draws.shape[0]
    n_q = draws.shape[1]
#   n_k = draws.shape[2]
    
    #matrix with all simulations
    simulations = np.zeros((n_q, n_r))
    
    #iterate over households
    for q in prange(n_q):
        rows = data[np.where(data[:,0] == q+1)]
        n_rows = len(rows)
        
        #iterate over draws per household
        for r in prange(n_r):
            probabilities= np.zeros(n_rows)
            
            #iterate over oberservations per househould
            for t in prange(n_rows):
                choices = np.zeros(4)
                
                #itetate over probability of choices per observation
                for j in prange(4):
                    utility = 0; #start with alpha
                    if j < 3: utility = c[j]
                    x = [rows[t][1+j], rows[t][5+j], rows[t][9+j]]
                    mu =  c[3:6]
                    sigma = exp(c[6:])
                    #np.exp(alpha +  np.dot(c[3:6],x) + np.dot(np.multiply(c[6:],draws[row[0]-1]),x))
                    for l in prange(3):
                          utility += mu[l] * x[l] + np.absolute(sigma[l]) * draws[r][q][l] * x[l]

                    choices[j] = np.exp(utility)
            
                probabilities[t] = choices[int(rows[t][13])] / np.sum(choices)
                
            simulations[q,r] = np.exp(np.log(probabilities).sum())
            
    estimates = np.zeros(n_q)
    for q in prange(n_q):
        estimates[q] = np.sum(simulations[q,:]) / n_r #.mean()
    
    res = -np.log(estimates).sum()
    
    if verbose == 2: print(res)
        
    return res

iterations = 1

   
def mixedlogit(data, drawtype, n_draws, c_0=False, method='BFGS', verbose=0):
    n_q = len(data.id.unique())
    
#     coefficients = [#alpha heinz41
#                     #alpha heinz32 
#                     #alpha heinz28
#                     #mu    display
#                     #mu    feat
#                     #mu    price
#                     #sigma dispay 
#                     #sigma feat 
#                     #sigma price 
#                     ]
    
    #generate inital values if neccesary
    if isinstance(c_0, bool) and c_0 == False:
            #genereate random starting coefficients
            c_0 = np.random.rand(9)
    elif len(c_0) != 9:
        raise Exception("Incorrect initial coefficients")
    
    #generate draws
    if drawtype == 'pseudo':
        draws = np.random.randn(n_draws, n_q, 3); 
    else:
         raise Exception("Incorrect Drawtype")
    
    
    global iterations
    iterations = 1
    def logging(xk):
        global iterations
        if(verbose >0):
            print("Iteration %d, coefficients:" % (iterations))
            print(xk)
            print('\n')
            iterations += 1
    
    start = time.time()
    res =  minimize(likelihood, c_0, args=(data.drop(columns='choice').values, draws, verbose), method=method, callback=logging)
    end = time.time()
    duration = end-start 
    
    #if verbose: 
    print("Optimization done, time elapsed: %s" % str(datetime.timedelta(seconds=round(duration))))
    
    res['duration'] = duration
    return res

benchmarking and comparing pandas and numba implementation

In [3]:
np.random.seed(1234)

#example run with BFGS
print(mixedlogit(df, drawtype="pseudo", n_draws=1, method='BFGS', verbose=1))

#example run with Nelder-Mead
#print(mixedlogit(df, drawtype="pseudo", n_draws=1, method='Nelder-Mead', verbose=False))

UnsupportedError: Failed in nopython mode pipeline (step: nopython mode backend)
[1mFailed in nopython mode pipeline (step: nopython frontend)
[1mUnsupported op-code encountered: arrayexpr(expr=(<built-in function eq>, [Var($50.12, <ipython-input-2-f67ad98b10bb> (20)), Var($50.15, <ipython-input-2-f67ad98b10bb> (20))]), ty=array(bool, 1d, F))
[1m
File "<ipython-input-2-f67ad98b10bb>", line 20:[0m
[1mdef likelihood(c, data, draws, verbose=False):
    <source elided>
    for q in prange(n_q):
[1m        rows = data[np.where(data[:,0] == q+1)]
[0m        [1m^[0m[0m
[0m
[0m[1m[1] During: lowering "id=15[LoopNest(index_variable = parfor_index.222, range = (0, $n_q.223, 1))]{50: <ir.Block at <ipython-input-2-f67ad98b10bb> (19)>}Var(parfor_index.222, <ipython-input-2-f67ad98b10bb> (19))" at <ipython-input-2-f67ad98b10bb> (19)[0m
Unsupported functionality was found in the code Numba was trying to compile.

If this functionality is important to you please file a feature request at

In [4]:
import pickle
dataset = pickle.load(open("datasets/n 10 - seed 1234 - time 2019-06-17-193522.pickle","rb"))

In [6]:
np.random.seed(4321)

c = [1.8, #alpha heinz41
     1.2, #alpha heinz32 
     2.0, #alpha heinz28
     1.3, #mu    display
     1.1, #mu    feat
     -1.8, #mu    price
     0.25, #sigma dispay 
     0.5, #sigma feat 
     1.2 #sigma price 
    ]

# c = np.multiply(c,1)

regressions = {}


for i, data in enumerate(dataset['data']):
#     try:
        print("regressing dataset "+str(i+1))
        print(mixedlogit(data, drawtype="pseudo", n_draws=500, method='BFGS', verbose=1))
#     except:
#         print("Something went wrong")
#     else:
#         print("Nothing went wrong")
        
np.save("500r",regressions)

regressing dataset 1
Iteration 1, coefficients:
[-0.06852486  1.06480582  0.6624098   0.37618077  0.22708767  0.33775352
  0.48597438  0.78405441  0.79613723]


Iteration 2, coefficients:
[0.14606819 0.52021475 3.03280112 1.76989777 0.91836505 0.43927571
 0.83523105 0.99653281 1.65158931]


Iteration 3, coefficients:
[1.15688612 1.60629098 3.79621817 2.84574568 1.56272282 1.15622552
 0.86644071 1.18135426 2.24148919]


Iteration 4, coefficients:
[1.76701767 1.33788385 3.75547917 2.84949895 1.69564189 1.33091534
 0.63368616 1.14350327 2.65693534]


Iteration 5, coefficients:
[ 2.36850906  1.67526569  4.58692831  1.98847864  1.86609821  1.35486431
 -0.50052228  0.5938128   3.08767691]


Iteration 6, coefficients:
[ 3.11026995  1.86866409  5.04562229  2.19894548  2.61100354  0.58646514
 -2.34436811 -0.13239032  2.75151389]


Iteration 7, coefficients:
[ 3.13319007  2.13235488  5.23619719  3.55472296  2.6179904   0.86643905
 -4.4930733  -3.4950753   3.21324648]


Iteration 8, coefficients:

ZeroDivisionError: division by zero

In [13]:
res = [ 44.52974914,  28.01881169,  50.01573162,  30.17899965,
        33.54040419, -40.38707385,  -6.6163476 , -18.71561041,
        30.09048101]

c = [1.8, #alpha heinz41
     1.2, #alpha heinz32 
     2.0, #alpha heinz28
     1.3, #mu    display
     1.1, #mu    feat
     -1.8, #mu    price
     0.25, #sigma dispay  
     0.5, #sigma feat 
     1.2 #sigma price 
    ]

regressions[1]

 duration: 5.635567665100098
      fun: 1405.1736168384664
 hess_inv: array([[ 1.12335867e-02,  5.59921069e-03,  9.58018239e-03,
         2.14884180e-04,  1.30605848e-03, -4.19124826e-03,
         5.37309924e-07,  1.84187838e-04, -2.44861050e-08],
       [ 5.59921069e-03,  7.21120609e-03,  7.03110843e-03,
         2.89801230e-03,  1.71514873e-03, -1.00825982e-03,
         4.99420632e-07, -4.32096624e-04,  2.97119246e-08],
       [ 9.58018239e-03,  7.03110843e-03,  1.38813428e-02,
         2.65533535e-03,  3.44111871e-03, -5.70891269e-03,
         4.02377286e-07, -4.53160954e-04, -8.58494499e-09],
       [ 2.14884180e-04,  2.89801230e-03,  2.65533535e-03,
         2.11257121e-02,  1.39207428e-03, -1.91276984e-04,
        -6.06013061e-07, -1.97965061e-05,  4.41683716e-07],
       [ 1.30605848e-03,  1.71514873e-03,  3.44111871e-03,
         1.39207428e-03,  1.51566744e-02, -2.65090523e-03,
         2.38314832e-07,  2.26923082e-04,  6.86561305e-08],
       [-4.19124826e-03, -1.00825982e-03

In [29]:
regression.x
np.multiply(regression.x, 1.8/regression.x[0])

array([ 1.80000000e+00,  1.21398172e+00,  2.02043310e+00,  3.08159860e+00,
        1.12572222e+00, -1.48027456e+00, -2.19695188e+00,  2.89043568e-04,
        1.28418895e+00])